In [1]:
from keras_vggface.vggface import VGGFace

Using TensorFlow backend.


In [2]:
from keras.layers import Flatten, Dense, Input, Reshape, Lambda
from keras.models import Model
from tensorflow.image import resize_images

In [3]:
img_inputs = Input(shape=(48*48*3,))
img_reshape = Reshape([48,48,3])(img_inputs)
img_resized = Lambda(lambda x : resize_images(x,[224,224],
    align_corners = True, # possibly important
    preserve_aspect_ratio = True))(img_reshape)
resize_model = Model(inputs=img_inputs, outputs=img_resized)

In [4]:
nb_class = 2
hidden_dim = 512
vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
last_layer = vgg_model.get_layer('pool3').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)
final_model = Model(inputs = resize_model.input, outputs = custom_vgg_model(resize_model.output))

58916864/58909280 [==============================] - 75s 1us/step


In [0]:
import numpy as np

In [16]:
final_model.compile('adam', 'binary_crossentropy')
final_model.fit(train_images_data, labels, validation_split=0.1, batch_size=512, epochs=3)

Train on 104541 samples, validate on 11616 samples
Epoch 1/3


ResourceExhaustedError: ignored

In [0]:
import matplotlib.pyplot as plt
plt.imshow(train_images_data.reshape((-1,56,56,3))[0])